<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="images/book_cover.jpg" width="120">

*Ce cahier contient un extrait de [Programmation Python et méthodes numériques - Un guide pour les ingénieurs et les scientifiques](https://pythonnumericalmethods.berkeley.edu/notebooks/Index.html), le contenu est également disponible sur [Berkeley Python Numerical Methods](https://pythonnumericalmethods.berkeley.edu/notebooks/Index.html).*

*Les droits d'auteur du livre appartiennent à Elsevier. Nous avons également ce livre interactif en ligne pour une meilleure expérience d'apprentissage. Le code est publié sous la [licence MIT](https://opensource.org/licenses/MIT). Si vous trouvez ce contenu utile, pensez à soutenir le travail sur [Elsevier](https://www.elsevier.com/books/python-programming-and-numerical-methods/kong/978-0-12-819549-9) ou [Amazon](https://www.amazon.com/Python-Programming-Numerical-Methods-Scientists/dp/0128195495/ref=sr_1_1?dchild=1&keywords=Python+Programming+and+Numerical+Methods+-+A+Guide+for+Engineers+and+Scientists&qid=1604761352&sr=8-1) !*

<!--NAVIGATION-->
< [11.4 JSON Files](chapter11.04-JSON-Files.ipynb) | [Contents](Index.ipynb) | [11.6 Summary and Problems](chapter11.06-Summary-and-Problems.ipynb) >

# Fichiers HDF5

En informatique scientifique, nous devons parfois stocker de grandes quantités de données avec un accès rapide, les formats de fichiers que nous avons introduits auparavant ne suffiront pas. Vous constaterez bientôt qu'il existe de nombreux cas, HDF5 (Hierarchical Data Format) est la solution. Il s'agit d'un format de données binaires puissant sans limite de taille de fichier. Il fournit des E/S parallèles (entrée/sortie) et effectue un certain nombre d'optimisations de bas niveau sous le capot pour rendre les requêtes plus rapides et les besoins de stockage réduits.

Un fichier HDF5 enregistre deux types d'objets : *ensembles de données*, qui sont des collections de données de type tableau (comme les tableaux NumPy), et *groupes*, qui sont des conteneurs de type dossier contenant des ensembles de données et d'autres groupes. Il existe également des attributs qui peuvent être associés aux ensembles de données et aux groupes pour décrire certaines propriétés. Ce qu'on appelle *hiérarchique* dans HDF5 fait référence au fait que les données peuvent être enregistrées comme un système de fichiers, avec des structures de type dossier, telles qu'un dossier, un sous-dossier (dans HDF5, cela est appelé groupe, sous-groupe). Les groupes fonctionnent comme des dictionnaires avec les *clés* et les *valeurs*, les *clés* sont les noms des groupes, et les *valeurs* sont les sous-groupes ou ensembles de données.

Afin d'utiliser HDF5 en lecture/écriture en Python, il existe certains packages ou wrappers pour répondre à ces objectifs. Les deux packages les plus courants sont [PyTables](https://www.pytables.org) et [h5py](https://www.h5py.org). Nous présenterons ici uniquement le h5py. Vous pouvez installer h5py en utilisant conda (j'espère que vous vous souvenez encore de comment faire cela, si vous oubliez, veuillez revenir au chapitre 1).

Après avoir installé h5py, vous pouvez suivre le guide de démarrage rapide dans la [documentation h5py](http://docs.h5py.org/en/latest/quick.html) pour démarrer rapidement. Mais ici, utilisons un exemple pour montrer comment créer et lire un fichier HDF5. Importons d'abord NumPy et h5py.

In [1]:
import numpy as np
import h5py

**Exemple :** Supposons que nous ayons déployé des instruments pour surveiller les accélérations et la localisation GPS dans la Bay Area, en Californie. Nous avons déployé deux accéléromètres à Berkeley et Oakland ainsi qu'une station GPS à San Francisco. Et ils enregistrent les données à différentes fréquences d'échantillonnage, l'accéléromètre de Berkeley échantillonnant les données toutes les 0,04 s et toutes les 0,01 s pour le capteur d'Oakland. Le GPS échantillonne l'emplacement toutes les 60 secondes à San Francisco. Nous voulons maintenant stocker les deux types de données dans un HDF5 ainsi que certains attributs indiquent où les données sont enregistrées, l'heure de début de l'enregistrement, le nom de la station et l'intervalle d'échantillonnage.

In [2]:
# Generate random data for recording
acc_1 = np.random.random(1000)
station_number_1 = '1'
# unix timestamp
start_time_1 = 1542000276
# time interval for recording
dt_1 = 0.04
location_1 = 'Berkeley'

acc_2 = np.random.random(500)
station_number_2 = '2'
start_time_2 = 1542000576
dt_2 = 0.01
location_2 = 'Oakland'

In [3]:
hf = h5py.File('station.hdf5', 'w')

In [4]:
hf['/acc/1/data'] = acc_1
hf['/acc/1/data'].attrs['dt'] = dt_1
hf['/acc/1/data'].attrs['start_time'] = start_time_1
hf['/acc/1/data'].attrs['location'] = location_1

hf['/acc/2/data'] = acc_2
hf['/acc/2/data'].attrs['dt'] = dt_2
hf['/acc/2/data'].attrs['start_time'] = start_time_2
hf['/acc/2/data'].attrs['location'] = location_2

hf['/gps/1/data'] = np.random.random(100)
hf['/gps/1/data'].attrs['dt'] = 60
hf['/gps/1/data'].attrs['start_time'] = 1542000000
hf['/gps/1/data'].attrs['location'] = 'San Francisco'

In [5]:
hf.close()

Le code ci-dessus montre les concepts de base de HDF5 : les groupes, les ensembles de données, les attributs. Nous créons d'abord un objet HDF5 pour l'écriture - station.hdf5. Ensuite, nous commençons à stocker les données dans différents groupes. Nous pouvons voir que nous avons deux groupes de niveau supérieur, à savoir acc et gps, tous deux contiennent des sous-groupes 1 ou 2 indiquant les noms des stations. Chaque station contiendra le sous-groupe de niveau suivant, les données, utilisé pour stocker les données du tableau que nous avons créées. Nous pourrions alors ajouter des attributs aux groupes ou aux données. Ici, nous avons uniquement ajouté *dt*, *start_time* et *location* comme attributs aux ensembles de données que nous stockons ici. Vous pouvez voir qu'elle est assez similaire à une structure de type dossier, avec les données *acc_1* enregistrées dans */acc/1/data*. Enfin, nous fermons l'objet fichier.

Nous pouvons maintenant voir que la sauvegarde des données dans HDF5 est facile et que nous pourrions utiliser les fonctions *create_dataset* et *create_group* comme indiqué dans le [démarrage rapide](http://docs.h5py.org /fr/latest/quick.html). Mais je préfère plutôt utiliser l’approche ci-dessus pour créer implicitement plusieurs groupes intermédiaires afin d’accéder à une structure de dossiers.

## Lire les fichiers HDF5

Supposons maintenant que vous envoyiez le *station.hdf5* à un collègue qui souhaite accéder aux données. Voici comment il procédera.

In [6]:
hf_in = h5py.File('station.hdf5', 'r')

In [7]:
list(hf_in.keys())

['acc', 'gps']

In [8]:
acc = hf_in['acc']

In [9]:
list(acc.keys())

['1', '2']

In [10]:
data_1 = hf_in['acc/1/data']

In [11]:
data_1.value[:10]

array([0.41820889, 0.89832446, 0.40229251, 0.41287538, 0.16173359,
       0.75855904, 0.89288185, 0.82944522, 0.84228139, 0.50365515])

In [12]:
list(data_1.attrs)

['dt', 'start_time', 'location']

In [13]:
data_1.attrs['dt']

0.04

In [14]:
data_1.attrs['location']

'Berkeley'

Nous pouvons voir que la lecture d'un HDF5 est également facile avec *h5py*. Après avoir lu le HDF5 dans *hf_in*, nous avons pu voir quels groupes se trouvent dans le HDF5 à l'aide de la fonction *keys*. Ensuite, nous pourrions accéder aux membres du groupe et voir ce que contiennent les sous-groupes comme *hf_in['acc']*, ou spécifier directement le chemin d'accès aux ensembles de données comme *hf_in['acc/1/ data']* et récupérez les données du tableau. Bien entendu, les attributs associés aux données pourraient également être accessibles sous forme de dictionnaire.

<!--NAVIGATION-->
< [11.4 JSON Files](chapter11.04-JSON-Files.ipynb) | [Contents](Index.ipynb) | [11.6 Summary and Problems](chapter11.06-Summary-and-Problems.ipynb) >